In [1]:
import psycopg2

In [2]:
try: 
    conn = psycopg2.connect(dbname= "mytable", user= "postgres", password="123456")
except psycopg2.Error as e:
    print(e)

In [5]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print(e)
conn.set_session(autocommit=True)

In [ ]:
# table name: customer_transactions
# column 0: customer id
#column 1: store id
#column 2: spent

# table name: customer
# c0: customer id
# c1: name
# c2: rewards

# table name: store
# c0: store id
# c1: state

# table name: items_purchased
# c0: customer id
# c1: item name

###  Creating table

#### Fact table

In [6]:
try:
    cur.execute('''
        CREATE TABLE IF NOT EXISTS customer_transactions(
            customer_id INTEGER,
            store_id INTEGER,
            spent NUMERIC
        )
    ''')
except psycopg2.Error as e:
    print(e)

# insert values
try:
    cur.execute('''
        INSERT INTO customer_transactions(customer_id, store_id, spent)
        VALUES 
        (1, 1, 20.50),
        (2, 1, 35.21)
    ''')
except psycopg2.Error as e:
    print(e)

#### Dimension table

In [20]:
try:
    cur.execute('''
        CREATE TABLE IF NOT EXISTS customer_1(
            customer_id INTEGER PRIMARY KEY,
            name VARCHAR(50),
            rewards BOOLEAN
        )
    ''')
except psycopg2.Error as e:
    print(e)

# insert values
try:
    cur.execute('''
        INSERT INTO customer_1(customer_id, name, rewards)
        VALUES
        (1, 'Amanda', True),
        (2, 'Toby', False)
    ''')
except psycopg2.Error as e:
    print(e)

In [11]:
try:
    cur.execute('''
        CREATE TABLE IF NOT EXISTS store(
            store_id INTEGER PRIMARY KEY,
            state VARCHAR(25)
        )
    ''')
except psycopg2.Error as e:
    print(e)

# insert
try:
    cur.execute('''
        INSERT INTO store(store_id, state)
        VALUES 
        (1,'CA'),
        (2, 'WA')
    ''')
except psycopg2.Error as e:
    print(e)


In [14]:
try:
    cur.execute('''
        CREATE TABLE items_purchased(
            customer_id INTEGER PRIMARY KEY,
            item_number INTEGER,
            item_name VARCHAR(50)
        )
    ''')
except psycopg2.Error as e:
    print(e)
    
#insert
try:
    cur.execute('''
        INSERT INTO items_purchased(customer_id, item_number, item_name)
        VALUES
        (1, 1, 'Rubber Soul'),
        (2, 3, 'Let it be')
    ''')
except psycopg2.Error as e:
    print(e)

### Show table

In [21]:
try:
    cur.execute('SELECT * FROM customer_transactions')
except psycopg2.Error as e:
    print(e)
print('Customer-transactions: ')
for i in cur.fetchall():
    print(i)

try:
    cur.execute('SELECT * FROM customer_1')
except psycopg2.Error as e:
    print(e)
print('\nCustomer: ')
for i in cur.fetchall():
    print(i)

try:
    cur.execute('SELECT * FROM store')
except psycopg2.Error as e:
    print(e)
print('\nStore: ')
for i in cur.fetchall():
    print(i)
    
try:
    cur.execute('SELECT * FROM items_purchased')
except psycopg2.Error as e:
    print(e)
print('\nItem: ')
for i in cur.fetchall():
    print(i)

Customer-transactions: 
(1, 1, Decimal('20.50'))
(2, 1, Decimal('35.21'))

Customer: 
(1, 'Amanda', True)
(2, 'Toby', False)

Store: 
(1, 'CA')
(2, 'WA')

Item: 
(1, 1, 'Rubber Soul')
(2, 3, 'Let it be')


#### Find all the customers that spent more than 30 dollars, who are they, which store they bought it from, location of the store, what they bought and if they are a rewards member.

In [27]:
try:
    cur.execute('''
        SELECT customer_1.customer_id ,customer_1.name, customer_1.rewards,store.store_id,
        store.state, items_purchased.item_name 
        FROM customer_transactions
        INNER JOIN store
        ON store.store_id = customer_transactions.store_id
        INNER JOIN customer_1
        ON customer_1.customer_id = customer_transactions.customer_id
        INNER JOIN items_purchased
        ON items_purchased.customer_id = customer_transactions.customer_id
        WHERE customer_transactions.spent > 30
    ''')
except psycopg2.Error as e:
    print(e)

for i in cur.fetchall():
    print(i)

(2, 'Toby', False, 1, 'CA', 'Let it be')


#### Query 2: How much did Customer 2 spend?

In [28]:
try:
    cur.execute('''
        SELECT customer_1.name , customer_transactions.spent FROM customer_1
        INNER JOIN customer_transactions
        ON customer_transactions.customer_id = customer_1.customer_id
        WHERE customer_1.customer_id = 2
    ''')
except psycopg2.Error as e:
    print(e)

for i in cur.fetchall():
    print(i)

('Toby', Decimal('35.21'))


### Close connection

In [29]:
cur.close()
conn.close()